In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import pickle
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define dataset paths
dataset_path = '../../'
train_path = os.path.join(dataset_path, "Dataset/Train")
val_path = os.path.join(dataset_path, "Dataset/Validation")
test_path = os.path.join(dataset_path, "Dataset/Test")

# Set a seed for reproducibility
seed = 42
torch.manual_seed(seed)

In [ ]:
# Apply FFT to each image in the data pipeline
def no_apply_fft(image):
    image_tensor = transforms.ToTensor()(image).unsqueeze(0)  # Convert to tensor and add batch dimension
    image_tensor.requires_grad = True
    return image_tensor.squeeze(0)

# Custom transformation to apply FFT in the data pipeline
class noFFTTransform:
    def __call__(self, image):
        return no_apply_fft(image)

In [ ]:
transformation_for_valntest_noFFT = transforms.Compose([
    transforms.Resize((224, 224)),
    noFFTTransform(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Directly normalize the tensor
])

In [ ]:
# Function to calculate recall and accuracy
def compute_metrics(outputs, labels):
    # Convert the logits to binary predictions
    predicted = (torch.sigmoid(outputs) > 0.5).float()  # Predictions as 0 or 1
    
    # True positives, false positives, false negatives, true negatives
    tp = torch.sum((predicted == 1) & (labels == 1)).item()  # True positives
    fp = torch.sum((predicted == 1) & (labels == 0)).item()  # False positives
    fn = torch.sum((predicted == 0) & (labels == 1)).item()  # False negatives
    tn = torch.sum((predicted == 0) & (labels == 0)).item()  # True negatives
    
    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    
    # Precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0  # Avoid division by zero
    
    # Recall
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # Avoid division by zero
    
    # F1-Score
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0  # Avoid division by zero
    
    return accuracy, recall, precision, f1

In [ ]:
# Load datasets
test_dataset = datasets.ImageFolder(root=test_path, transform=transformation_for_valntest_noFFT)

# DataLoader with batch size
batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

<h1>Evaluation</h1>

In [ ]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        # Max pool after conv1
        self.pool1 = nn.MaxPool2d(2, 2)  # 2x2 max pooling
        
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        # Max pool after conv2
        self.pool2 = nn.MaxPool2d(2, 2)  # 2x2 max pooling

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Adjusted based on the size after pooling
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        # Pass input through first convolutional layer
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)  # Apply max pooling after conv1
        
        # Pass output of first conv layer through second convolutional layer
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)  # Apply max pooling after conv2

        # Flatten output of second conv layer
        x = x.view(x.size(0), -1)  # Flatten the tensor
        # Pass flattened output through first fully connected layer
        x = self.fc1(x)
        x = F.relu(x)
        # Pass output of first fully connected layer through second fully connected layer
        x = self.fc2(x)
        return x


In [ ]:
# Root directory containing the model folders
root_path = "../models/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MNIST_CNN()
criterion = nn.BCEWithLogitsLoss()

# Iterate over the folders in the root directory
for saved_model in os.listdir(root_path):
    model = MNIST_CNN()
    model = model.to(device)
    model.eval()
    evaluation_metrics=[]
    # Check if the file ends with .pth
    if saved_model.endswith(".pth"):
        # Define the path for the pickle file
        pickle_filename = os.path.splitext(saved_model)[0] + "_evaluation.pkl"
        pickle_file_path = os.path.join(root_path, pickle_filename)
        
        # Check if the pickle file already exists
        if os.path.exists(pickle_file_path):
            print(f"Loading the pickle file: {pickle_file_path}")
            with open(pickle_file_path, 'rb') as f:
                evaluation_metrics = pickle.load(f)

        else:
            print(f"Loading the file: {saved_model}")
            model_path=os.path.join(root_path,saved_model)
            model.load_state_dict(torch.load(model_path, map_location=device))
            model.eval()
            model = model.to(device)
            criterion = nn.BCEWithLogitsLoss()
            # Testing loop
            correct = 0
            total = 0
            running_loss = 0.0
            true_positive = 0
            false_positive = 0
            false_negative = 0
            true_negative = 0
            all_preds = []
            all_labels = []
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
                    # Get model predictions
                    outputs = model(inputs).squeeze()  # Get model output
                    labels = labels.view(-1)
    
                    # Compute loss
                    loss = criterion(outputs, labels)
                    running_loss += loss.item()
                    # Compute metrics manually
                    predicted = (torch.sigmoid(outputs) > 0.5).float()  # Convert to binary predictions
                    total += labels.size(0)
                    # Update counts for recall and precision
                    true_positive += ((predicted == 1) & (labels == 1)).sum().item()
                    false_positive += ((predicted == 1) & (labels == 0)).sum().item()
                    false_negative += ((predicted == 0) & (labels == 1)).sum().item()
                    true_negative += ((predicted == 0) & (labels == 0)).sum().item()
                    all_preds.extend(predicted.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
            correct = true_positive + true_negative
            # Compute metrics manually
            avg_accuracy = correct / total
            avg_recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
            avg_precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
            avg_f1 = 2 * (avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0
            # Collect the evaluation metrics for this epoch
            epoch_metrics = {
                "Loss": running_loss,
                "Accuracy": avg_accuracy,
                "Precision": avg_precision,
                "Recall": avg_recall,
                "F1 Score": avg_f1,
                "True Positive": true_positive,
                "True Negative": true_negative,
                "False Positive": false_positive,
                "False Negative": false_negative
            }
            evaluation_metrics.append(epoch_metrics)
            
            # Save evaluation metrics as a pickle file
            with open(pickle_file_path, 'wb') as f:
                pickle.dump(evaluation_metrics, f)
            print(f"Saved evaluation results to: {pickle_file_path}")
    
        # After processing all weight files (epochs), create a DataFrame
        df = pd.DataFrame(evaluation_metrics)
        # Save the DataFrame to an Excel file (one per folder)
        if not os.path.exists("../excel folder/"):
            os.makedirs("../excel folder/")
        excel_file=f"../excel folder/{saved_model}_evaluation.xlsx"
        df.to_excel(excel_file, index=False)
        print(f"Saved evaluation results to: {excel_file}")
